The goal of the code below is to use topic modeling to investigate the relationship between some philosophical texts. In a previous step, I scraped them up from the Gutenberg project and stored them locally as text files.

To start building the topic model - load in some libraries, including numpy and sklearn.

In [1]:
import glob, sys, string
import numpy as np
from sklearn import feature_extraction, decomposition

Define some functions, where we'll fit the model and investigate the output.

In [2]:
def print_top_words(model, feature_names, n_top_words):
        for topic_idx, topic in enumerate(model.components_):
                print("Topic #%d:" % topic_idx)
                print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))

def topic_modeling_prep(filenames, n_features):
        vectorizer = feature_extraction.text.CountVectorizer(input='filename', max_df=0.95, min_df=2, max_features=n_features, stop_words='english')
        tf_matrix = vectorizer.fit_transform(filenames)
        tf_array = tf_matrix.toarray()
        return vectorizer, tf_matrix, tf_array

def calc_lda(tf_array, n_topics=10,max_iter=20):
        lda = decomposition.LatentDirichletAllocation(n_topics=n_topics, max_iter=max_iter,
                                learning_method='online',
                                learning_offset=50.)
        doctopic = lda.fit_transform(tf_array)
        return doctopic, lda

def assign_topic(text_names,doctopic,num_topics):
        doctopic_grouped = np.zeros((text_names.shape[0], num_topics))
        for i, name in enumerate(text_names):
                doctopic_grouped[i, :] = np.mean(doctopic[text_names == name, :], axis=0)
        doc_topic_assign = np.argmax(doctopic_grouped, axis=1)
        return doc_topic_assign

def summarize_results(doc_topic_assign, lda, vectorizer, filenames):
        summary = read_summary()
        bookids = [f[5:-4] for f in filenames]

        # for each topic, print out words and documents.
        feature_names = vectorizer.get_feature_names()
        n_top_words = 5
        for topic_idx, topic in enumerate(lda.components_):
                print "Topic #%d:" % topic_idx,
                print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))

                # which docs are placed in this topic?
                ids = np.where(doc_topic_assign==topic_idx)[0].tolist()
                print("\n".join([summary[(bookids[i],'title')] + " by " + summary[(bookids[i],'author')]  for i in ids]))
                print("\n")

def read_summary():
        # read in summary file with title/author/year info
        summary = {}
        with open('data/summary.txt','r') as f1:
                for line in f1:
                        cols                            = line.split('\t')
                        summary[(str(cols[0]),'title')] = str(cols[1])
                        summary[(str(cols[0]),'author')]= str(cols[2])
                        summary[(str(cols[0]),'year')]  = str(cols[3]).rstrip()
        return summary


Now let's run it! This step takes a few seconds. Then below is the output by topic.

In [3]:
# constants
num_topics = 10
num_features = 1000

# read input files
filenames = glob.glob("data/[0-9]*.txt")
text_names = np.asarray(filenames)

# create model
vectorizer, tf_matrix, tf_array = topic_modeling_prep(filenames,num_features)
doctopic, lda = calc_lda(tf_array,num_topics)
doc_topic_assign = assign_topic(text_names,doctopic,num_topics)

# describe results
summarize_results(doc_topic_assign, lda, vectorizer,filenames)


Topic #0: moral principle beauty human object
The Aesthetical Essays by Frederich Schiller
Literary and Philosophical Essays by  Various
Beyond Good and Evil by Friedrich Nietzsche
Utilitarianism by John Stuart Mill
Lectures on the true, the beautiful and the good by Victor Cousin
The Philosophical Letters by Frederich Schiller
The Philosophy of the Moral Feelings by John Abercrombie


Topic #1: greek letter small experience existence
Poetics by Aristotle
The Critique of Pure Reason by Immanuel Kant
Ontology or the Theory of Being by Peter Coffey


Topic #2: government people socrates law opinion
Critical Miscellanies, Vol. 3 (of 3) by John Morley
Apology by Plato
Euthyphro by Plato
The Poetics by Aristotle
The Queen's Matrimonial Ladder by William Hone
Beyond Good and Evil by Friedrich Nietzsche
The English Utilitarians, Volume I. by Leslie Stephen
Second Treatise of Government by John Locke
Considerations on Representative Government by John Stuart Mill
The Republic by Plato
Ion by P

That's pretty cool! Some of the groups almost make sense! Is there a way to visualize this?

In [4]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [5]:
pyLDAvis.sklearn.prepare(lda,tf_matrix, vectorizer)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
8      18.233811        1       1 -0.090355  0.034829
5      14.896201        1       2  0.055881 -0.006186
2      12.351360        1       3  0.049747  0.081599
4       9.794357        1       4  0.035776 -0.035959
1       9.589556        1       5 -0.056741 -0.099458
0       8.860858        1       6  0.039882 -0.009796
7       8.188870        1       7 -0.051526 -0.011907
3       6.729335        1       8 -0.028295 -0.000158
6       6.228704        1       9  0.094973 -0.019610
9       5.126948        1      10 -0.049341  0.066647, topic_info=     Category         Freq          Term        Total  loglift  logprob
term                                                                   
415   Default  6690.000000           god  6690.000000  30.0000  30.0000
544   Default  4701.000000        letter  4701.000000  29.0000  29.0000
422   Default  5730.000000         greek  5730.000000  28.0000  28.0000
859   Default  5518.000000         small  5518.000000  27.0000  27.0000
931   Default  1507.000000          thou  1507.000000  26.0000  26.0000
386   Default  2498.000000      footnote  2498.000000  25.0000  25.0000
600   Default  4367.000000         moral  4367.000000  24.0000  24.0000
754   Default  3140.000000   proposition  3140.000000  23.0000  23.0000
755   Default  2507.000000  propositions  2507.000000  22.0000  22.0000
320   Default  1554.000000           est  1554.000000  21.0000  21.0000
829   Default  1276.000000           sec  1276.000000  20.0000  20.0000
463   Default  5103.000000         ideas  5103.000000  19.0000  19.0000
460   Default  5799.000000          idea  5799.000000  18.0000  18.0000
420   Default  2523.000000    government  2523.000000  17.0000  17.0000
115   Default   749.000000         burke   749.000000  16.0000  16.0000
818   Default   787.000000            sa   787.000000  15.0000  15.0000
988   Default  7599.000000         world  7599.000000  14.0000  14.0000
630   Default  6076.000000        object  6076.000000  13.0000  13.0000
624   Default  1871.000000           non  1871.000000  12.0000  12.0000
862   Default  2103.000000      socrates  2103.000000  11.0000  11.0000
632   Default  4617.000000       objects  4617.000000  10.0000  10.0000
687   Default  6215.000000    philosophy  6215.000000   9.0000   9.0000
128   Default  6940.000000         cause  6940.000000   8.0000   8.0000
994   Default   703.000000            ye   703.000000   7.0000   7.0000
999   Default   651.000000   zarathustra   651.000000   6.0000   6.0000
860   Default  2941.000000        social  2941.000000   5.0000   5.0000
868   Default  3329.000000          soul  3329.000000   4.0000   4.0000
88    Default   858.000000         bacon   858.000000   3.0000   3.0000
105   Default  1074.000000       bergson  1074.000000   2.0000   2.0000
256   Default  3159.000000           did  3159.000000   1.0000   1.0000
...       ...          ...           ...          ...      ...      ...
754   Topic10   642.923932   proposition  3140.781212   1.3845  -4.5969
921   Topic10   398.667706          term  1971.142867   1.3724  -5.0748
562   Topic10   136.794738          lord   681.950376   1.3642  -6.1445
336   Topic10   150.652753      examples   752.686404   1.3620  -6.0480
389   Topic10   126.866654        formal   637.291839   1.3566  -6.2198
456   Topic10   150.501961         house   802.005915   1.2975  -6.0490
722   Topic10   121.278593      premises   652.069420   1.2886  -6.2649
744   Topic10   148.097572    production   798.168695   1.2862  -6.0651
769   Topic10   204.506952      quantity  1106.473437   1.2823  -5.7424
279   Topic10   142.813183      division   775.093101   1.2792  -6.1014
959   Topic10   237.209712          used  1429.080847   1.1748  -5.5940
492   Topic10   239.585558     inference  1449.202977   1.1708  -5.5840
153   Topic10   402.789196         class  2775.472085   1.0405  -5.06